### see acoustic_multichannel for downloading data (migrate to README later)

In [4]:
import tensorflow as tf
import numpy as np

import sys
import os

sys.path.insert(0, 'models/research/audioset/vggish/')

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim
from sklearn.model_selection import train_test_split

In [5]:
# Charger le modèle VGGish
vgg_graph = tf.Graph()
with vgg_graph.as_default():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess, 'models/research/audioset/vggish//vggish_model.ckpt')

/opt/mamba/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/opt/mamba/lib/python3.11/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


INFO:tensorflow:Restoring parameters from models/research/audioset/vggish//vggish_model.ckpt


2024-04-23 16:43:06.110628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [6]:
# Charger les données d'entrée (RIRs) et de sortie (positions reconnues) depuis les fichiers .npy
input_data = np.load('data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy')
output_data = np.load('data/LivingRoom_preprocessed_hack/Human1/centroid.npy')

# Diviser les données en ensembles d'entraînement et de test
train_input, test_input, train_output, test_output = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

In [12]:
# Extraction des caractéristiques avec VGGish
def extract_vggish_features(data):
    features = []
    with vgg_graph.as_default():
        for rir in data:
            # Prétraitement des RIRs
            # Ici, vous devriez adapter le prétraitement en fonction de vos données RIRs
            # Par exemple, redimensionnement, normalisation, etc.
            # Ensuite, convertissez les RIRs en spectrogrammes
            spectrogram = vggish_input.waveform_to_examples(rir, vggish_params.SAMPLE_RATE)
            # Extraction des caractéristiques avec VGGish
            features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
            embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)
            embedding = sess.run(embedding_tensor, feed_dict={features_tensor: spectrogram})
            features.append(embedding)
    return np.array(features)

In [13]:
train_input.shape

(800, 4, 667200)

In [14]:
# Extraction des caractéristiques pour les ensembles d'entraînement et de test
# train_features = extract_vggish_features(train_input)
# test_features = extract_vggish_features(test_input)